In [2]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [20]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("C:\\Users\\Aman\\Desktop\\rag_app\\Aman_kumar_Singh_Resume.pdf")
docs = loader.load()

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap = 40)
final_docs = text_splitter.split_documents(docs)

In [34]:
for i, doc in enumerate(final_docs[:5]):
    print(doc.page_content[:300])

AMAN  KUMAR  SINGH
Kolkata,  West  Bengal     |     7004803067     |      amanksingh951@gmail.com     |
amanksingh951@gmail.com     |       linkedin.com/amanksingh951
Overview  A  result  oriented  Data  Science  professional  with  hands-on  experience  in  Azure
with  hands-on  experience  in  Azure  ML,  Azure  AI  services,  LLMs,  RAG  applications  and


In [23]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding_key = os.getenv("HUGGINGFACEHUB_API_TOKEN")

embeddings = HuggingFaceEmbeddings(model_name = 'all-MiniLM-L6-v2')

In [39]:
from  langchain_community.vectorstores import Chroma
vectordb = Chroma.from_documents(final_docs,embeddings)

In [40]:
from langchain_cohere import ChatCohere
cohere_api_key = os.getenv('API_KEY')
llm = ChatCohere(cohere_api_key=cohere_api_key)

In [63]:
from langchain_core.prompts import ChatPromptTemplate

def get_resume_system_prompt():
    return """You are a highly knowledgeable resume making specialist.
Answer the user's question using only the provided context about resume.
If the context is not relevant, say "I don't have enough information."
Be concise, factual, and avoid speculation."""

# Use the function to build the prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", get_resume_system_prompt()),
    ("human", "Question: {input}\n\nContext:\n{context}\n\nAnswer in plain English:")
])

In [64]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectordb.as_retriever(search_kwargs = {'k':3})

In [65]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [66]:
parsed_docs = document_chain | parser

In [67]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever,parsed_docs)

In [68]:
query = "suggest how to make the resume better"
response = retrieval_chain.invoke({"input":query})

In [69]:
print(response['answer'])

To improve your resume, consider the following suggestions:

1. **Add specific achievements**: Under your experience section, include measurable accomplishments or projects you completed as a Developer at Cognizant. For example, "Developed and deployed a GenAI-based application that improved [specific metric or outcome]."

2. **Expand on skills**: List technical skills relevant to your role, such as programming languages, frameworks, or tools you're proficient in. Since you have a keen interest in GenAI, highlight any related skills or experience.

3. **Format education section**: Reformat the education section to be more concise and clear. For example:
   - **Education**: Trident Academy of Technology, 2018-2022 | Bachelor's in Information Technology

4. **Add a summary or objective**: Include a brief summary or objective statement at the top of your resume to highlight your career goals and key strengths.

5. **Use consistent formatting**: Ensure consistent formatting throughout the 